In [120]:
import tensorflow as tf
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.misc
import glob
import sys

In [121]:
def get_img_array(path):
    """
    Given path of image, returns it's numpy array
    """
    return scipy.misc.imread(path)

def get_files(folder):
    """
    Given path to folder, returns list of files in it
    """
    filenames = [file for file in glob.glob(folder+'*/*')]
    filenames.sort()
    return filenames

def get_label(filepath, label2id):
    """
    Files are assumed to be labeled as: /path/to/file/999_frog.png
    Returns label for a filepath
    """
    tokens = filepath.split('/')
    label = tokens[-1].split('_')[1][:-4]
    if label in label2id:
        return label2id[label]
    else:
        sys.exit("Invalid label: " + label)

In [122]:
def get_labels(folder, label2id):
    """
    Returns vector of labels extracted from filenames of all files in folder
    :param folder: path to data folder
    :param label2id: mapping of text labels to numeric ids. (Eg: automobile -> 0)
    """
    files = get_files(folder)
    y = []
    for f in files:
        y.append(get_label(f,label2id))
    return np.array(y)

def one_hot(y, num_classes=10):
    """
    Converts each label index in y to vector with one_hot encoding
    """
    y_one_hot = np.zeros((num_classes, y.shape[0]))
    y_one_hot[y, range(y.shape[0])] = 1
    return y_one_hot

def get_label_mapping(label_file):
    """
    Returns mappings of label to index and index to label
    The input file has list of labels, each on a separate line.
    """
    with open(label_file, 'r') as f:
        id2label = f.readlines()
        id2label = [l.strip() for l in id2label]
    label2id = {}
    count = 0
    for label in id2label:
        label2id[label] = count
        count += 1
    return id2label, label2id

def get_images(folder):
    """
    returns numpy array of all samples in folder
    each column is a sample resized to 30x30 and flattened
    """
    files = get_files(folder)
    images = []
    count = 0

    for f in files:
        count += 1
        if count % 10000 == 0:
            print("Loaded {}/{}".format(count,len(files)))
        img_arr = get_img_array(f)
        img_arr = img_arr.flatten() / 255.0
        images.append(img_arr)
    X = np.column_stack(images)

    return X

def get_train_data(data_root_path):
    """
    Return X and y
    """
    train_data_path = data_root_path + 'train'
    id2label, label2id = get_label_mapping(data_root_path+'labels.txt')
    print(label2id)
    X = get_images(train_data_path)
    y = get_labels(train_data_path, label2id)
    return X, y

def save_predictions(filename, y):
    """
    Dumps y into .npy file
    """
    np.save(filename, y)

In [123]:
cd ~/tensorflow

/Users/danielfirst/tensorflow


In [160]:
# Load the data
data_root_path = 'cifar10-hw1/'
X_train_total, y_train_total = get_train_data(data_root_path) # this may take a few minutes
X_test = get_images(data_root_path + 'test')
print('Data loading done')

{'airplane': 0, 'automobile': 1, 'bird': 2, 'cat': 3, 'deer': 4, 'dog': 5, 'frog': 6, 'horse': 7, 'ship': 8, 'truck': 9}
Loaded 10000/50000
Loaded 20000/50000
Loaded 30000/50000
Loaded 40000/50000
Loaded 50000/50000
Loaded 10000/10000
Data loading done


In [166]:
X_train=X_train_total[:,:45000]
X_val=X_train_total[:,45000:]
y_train=y_train_total[:45000]
y_val=y_train_total[45000:]

In [169]:
def one_hot(y, num_classes=10):
    """
    Converts each label index in y to vector with one_hot encoding
    """
    y_one_hot = np.zeros((num_classes, y.shape[0]))
    y_one_hot[y, range(y.shape[0])] = 1
    return y_one_hot
print(y_train.shape)
y_train=one_hot(y_train)
y_val=one_hot(y_val)
print(y_train.shape)

(10, 45000)


IndexError: arrays used as indices must be of integer (or boolean) type

In [171]:
def get_batch(X_train,y_train,batch_size=100):
    ix = np.random.choice(X_train.shape[1], batch_size, replace=True)
    X_train_batch=X_train[:,ix]
    y_train_batch=y_train[:,ix]
    return X_train_batch.T,y_train_batch.T

In [172]:
# from tensorflow.examples.tutorials.mnist import input_data
# mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

In [173]:
sess = tf.InteractiveSession()

In [174]:
x = tf.placeholder(tf.float32, shape=[None, 3072])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

In [175]:
W = tf.Variable(tf.zeros([3072,10]))
b = tf.Variable(tf.zeros([10]))
sess.run(tf.global_variables_initializer())
y = tf.matmul(x,W) + b
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

In [176]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [177]:
for _ in range(1000):
#   batch = mnist.train.next_batch(100)
  batch_x,batch_y=get_batch(X_train,y_train)
  train_step.run(feed_dict={x: batch_x, y_: batch_y})

In [179]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

In [180]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [189]:
print(accuracy.eval(feed_dict={x: X_val.T, y_: y_val.T}))

0.2606


We thus have a .92 accuracy without convolution. How much will convolution improve our model?

In [202]:
X_train.shape

(3072, 45000)

In [193]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

In [194]:
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [208]:
#first conv layer
W_conv1 = weight_variable([5, 5, 3, 32])
b_conv1 = bias_variable([32])
#x_image = tf.reshape(x, [-1, 28, 28, 1])
x_image = tf.reshape(x, [-1, 32, 32, 3])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

#second conv layer
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

#dense layer
W_fc1 = weight_variable([8 * 8 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 8*8*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

#dropout for second part
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

#readout
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [ ]:
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
#train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  for i in range(10000):
#    batch = mnist.train.next_batch(50)
    batch_x,batch_y = get_batch(X_train,y_train,batch_size=1000)
    if i % 100 == 0:
      train_accuracy = accuracy.eval(feed_dict={
          x: batch_x, y_: batch_y, keep_prob: 1.0})
      print('step %d, training accuracy %g' % (i, train_accuracy))
    train_step.run(feed_dict={x: batch_x, y_: batch_y, keep_prob: 1.0})

  print('test accuracy %g' % accuracy.eval(feed_dict={x: X_val.T, y_: y_val.T, keep_prob: 1.0}))

step 0, training accuracy 0.097
step 100, training accuracy 0.093
step 200, training accuracy 0.098
step 300, training accuracy 0.087
step 400, training accuracy 0.094
step 500, training accuracy 0.103
step 600, training accuracy 0.111
step 700, training accuracy 0.097
step 800, training accuracy 0.098
step 900, training accuracy 0.092
step 1000, training accuracy 0.1
step 1100, training accuracy 0.106
step 1200, training accuracy 0.1
step 1300, training accuracy 0.089
step 1400, training accuracy 0.099
step 1500, training accuracy 0.105
step 1600, training accuracy 0.118
step 1700, training accuracy 0.1
step 1800, training accuracy 0.097
step 1900, training accuracy 0.118
step 2000, training accuracy 0.114
step 2100, training accuracy 0.097
step 2200, training accuracy 0.099
step 2300, training accuracy 0.101
step 2400, training accuracy 0.102
step 2500, training accuracy 0.105
step 2600, training accuracy 0.113
step 2700, training accuracy 0.093
step 2800, training accuracy 0.111
ste